In [156]:
import mlflow
import pandas as pd
import prophet

### Prepare new data to update models

In [157]:
new_data = pd.read_csv('./data/NASDAQ_100/AAPL.csv')
new_data['Date'] = pd.to_datetime(new_data['Date'])
# latest_date = df_data['Date'].tail(6).values[3]
# latest_date = ''
# with open('./containers/update/latest_date.txt', 'r') as f:
#     latest_date = f.readline()
#     f.close()
# latest_date = pd.to_datetime(latest_date)
# new_data = df_data[df_data['Date'] > latest_date]
new_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-19,170.000000,171.080002,165.940002,166.229996,165.270813,94815000
1,2022-01-20,166.979996,169.679993,164.179993,164.509995,163.560730,91420500
2,2022-01-21,164.419998,166.330002,162.300003,162.410004,161.472855,122848900
3,2022-01-24,160.020004,162.300003,154.699997,161.619995,160.687393,162294600
4,2022-01-25,158.979996,162.759995,157.020004,159.779999,158.858032,115798400
...,...,...,...,...,...,...,...
247,2023-01-12,133.880005,134.259995,131.440002,133.410004,133.410004,71379600
248,2023-01-13,132.029999,134.919998,131.660004,134.759995,134.759995,57758000
249,2023-01-17,134.830002,137.289993,134.130005,135.940002,135.940002,63646600
250,2023-01-18,136.820007,138.610001,135.029999,135.210007,135.210007,69567000


### Updating models

In [158]:
MLFLOW_IP = '91.206.15.104'
MLFLOW_PASSWORD = 'lH19Y0ImXQNg'
MLFLOW_USER = 'godder'
MLFLOW_PORT = '9899'

In [159]:
mlflow.set_tracking_uri('http://{}:{}'.format(MLFLOW_IP, MLFLOW_PORT))

In [160]:
tracking_uri = mlflow.get_tracking_uri()
registry_uri = mlflow.get_registry_uri()

client = mlflow.MlflowClient(tracking_uri=tracking_uri, registry_uri=registry_uri)

In [161]:
ticker_name = 'AAPL'

#### Prophet

In [162]:
# experiment_id = mlflow.create_experiment(
#     "Prophet update",
#     artifact_location="sftp://{}:{}@{}:/home/godder/mlflow_storage/artifacts".format(MLFLOW_USER, MLFLOW_PASSWORD, MLFLOW_IP),
#     tags={"version": ".1"},
# )

In [163]:
vers = []
for mv in client.search_model_versions("name='Prophet'"):
    vers.append(int(mv.version))
prophet_latest = max(vers)

In [164]:
exp_id = dict(mlflow.get_experiment_by_name("Prophet update"))['experiment_id']

In [165]:
mlflow.set_experiment(experiment_id=exp_id)
print("Set experiment")

Set experiment


In [166]:
mlflow.start_run(experiment_id=exp_id, run_name='Prophet updating')

<ActiveRun: >

In [167]:
old_model = mlflow.prophet.load_model(
    model_uri=f"models:/Prophet/{prophet_latest}"
)

/usr/local/lib/python3.8/dist-packages/prophet/serialize.py:160: FutureWarning: The behavior of Timestamp.utcfromtimestamp is deprecated, in a future version will return a timezone-aware Timestamp with UTC timezone. To keep the old behavior, use Timestamp.utcfromtimestamp(ts).tz_localize(None). To get the future behavior, use Timestamp.fromtimestamp(ts, 'UTC')
  setattr(model, attribute, pd.Timestamp.utcfromtimestamp(model_dict[attribute]).tz_localize(None))


In [168]:
updated_model = prophet.Prophet(changepoint_prior_scale=old_model.changepoint_prior_scale, seasonality_prior_scale=old_model.seasonality_prior_scale, weekly_seasonality=old_model.weekly_seasonality)
new_df = new_data[['Date', 'Close']].copy()
new_df.rename({'Date': 'ds', 'Close': 'y'}, axis=1, inplace=True)
updated_model.fit(new_df)

20:33:15 - cmdstanpy - INFO - Chain [1] start processing
20:33:16 - cmdstanpy - INFO - Chain [1] done processing


In [169]:
future = updated_model.make_future_dataframe(periods=10)
signature = mlflow.models.signature.infer_signature(future)

In [170]:
mlflow.prophet.log_model(updated_model,
                         'prophet',
                         registered_model_name='Prophet',
                         signature=signature,
                         input_example=future)

Registered model 'Prophet' already exists. Creating a new version of this model...
2023/01/19 20:33:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Prophet, version 19
Created version '19' of model 'Prophet'.


In [171]:
mlflow.end_run()

In [172]:
version = prophet_latest + 1
client.transition_model_version_stage(
    name="Prophet",
    version=version,
    stage="Staging"
)

for ver in range(1, version):
    client.transition_model_version_stage(
        name="Prophet",
        version=ver,
        stage="Archived"
    )
print('done')

done


#### Transformer

In [ ]:
experiment_id = mlflow.create_experiment(
    "Transformer update",
    artifact_location="sftp://{}:{}@{}:/home/godder/mlflow_storage/artifacts".format(MLFLOW_USER, MLFLOW_PASSWORD, MLFLOW_IP),
    tags={"version": ".1"},
)

In [ ]:
exp_id = dict(mlflow.get_experiment_by_name("Transformer update"))['experiment_id']

In [ ]:
mlflow.set_experiment(experiment_id=exp_id)
print("Set experiment")

In [ ]:
version = ####
client.transition_model_version_stage(
    name="CompatibleTransformer",
    version=version,
    stage="Staging"
)

for ver in range(1, version):
    client.transition_model_version_stage(
        name="CompatibleTransformer",
        version=ver,
        stage="Archived"
    )
print('done')

#### ARIMA

In [173]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import pmdarima

In [174]:
# experiment_id = mlflow.create_experiment(
#     "ARIMA update",
#     artifact_location="sftp://{}:{}@{}:/home/godder/mlflow_storage/artifacts".format(MLFLOW_USER, MLFLOW_PASSWORD, MLFLOW_IP),
#     tags={"version": ".1"},
# )

In [175]:
vers = []
for mv in client.search_model_versions("name='ARIMA'"):
    vers.append(int(mv.version))
arima_latest = max(vers)

In [176]:
exp_id = dict(mlflow.get_experiment_by_name("ARIMA update"))['experiment_id']

In [177]:
mlflow.set_experiment(experiment_id=exp_id)
print("Set experiment")

Set experiment


In [178]:
mlflow.start_run(experiment_id=exp_id, run_name='ARIMA updating')

<ActiveRun: >

In [179]:
new_df  = new_data[['Date','Close']].copy()
new_df['Close'] = np.log(new_df['Close'])

In [180]:
old_model = mlflow.pmdarima.load_model(
    model_uri=f"models:/ARIMA/{arima_latest}"
)

In [181]:
new_model = ARIMA(new_df['Close'], order=old_model.order)
updated_model = new_model.fit()

In [182]:
m = pmdarima.auto_arima(new_df['Close'])

In [183]:
signature = mlflow.models.signature.infer_signature(pd.DataFrame([{"n_periods": 10, "return_conf_int": True, "alpha": 0.1}]))

/usr/local/lib/python3.8/dist-packages/mlflow/models/signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)


In [184]:
mlflow.pmdarima.log_model(m, 'ARIMA', registered_model_name='ARIMA',
                         signature=signature,
                         input_example=pd.DataFrame([{"n_periods": 10, "return_conf_int": True, "alpha": 0.1}]))

Registered model 'ARIMA' already exists. Creating a new version of this model...
2023/01/19 20:33:31 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ARIMA, version 9
Created version '9' of model 'ARIMA'.


In [185]:
mlflow.end_run()

In [186]:
version = arima_latest + 1
client.transition_model_version_stage(
    name="ARIMA",
    version=version,
    stage="Staging"
)

for ver in range(1, version):
    client.transition_model_version_stage(
        name="ARIMA",
        version=ver,
        stage="Archived"
    )
print('done')

done
